target: 'Best Performance'  
split 70-30, random state: 2020

1. Bersihkan data  
2. Buat analisa singkat hubungan antara fitur dan target 
3. Buat skema preprosessing   
4. Pilih fitur menggunakan  :
    * univariate statistics
    * iterative feature selection  

Gunakan model:  
* Decision Tree Classifier
* Logistic Regression

Bandingkan performa model dan fitur yang terpilih.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('BRI.csv')
df.head()

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,Employee_status,gender,age,marital_status_maried(Y/N),...,assign_of_otherposition,annual leave,sick_leaves,Best Performance,Avg_achievement_%,Last_achievement_%,Achievement_above_100%_during3quartal,achievement_target_1,achievement_target_2,achievement_target_3
0,JG05,8.00,PG06,8.00,0.50,RM_type_A,Permanent,Female,1964,Y,...,0,8,0,0,68.85,72.53,0.0,achiev_50%-100%,achiev_< 50%,reached
1,JG04,5.17,PG04,2.00,1.25,RM_type_A,Permanent,Male,1979,Y,...,9,3,0,0,94.81,101.13,3.0,achiev_50%-100%,achiev_50%-100%,reached
2,JG04,1.83,PG03,1.83,1.50,RM_type_A,Permanent,Male,1980,Y,...,0,6,0,0,72.82,71.30,0.0,achiev_100%-150%,achiev_< 50%,not_reached
3,JG04,7.92,PG04,4.00,0.17,RM_type_A,Permanent,Male,1984,Y,...,0,3,0,0,86.89,90.09,2.0,achiev_50%-100%,achiev_50%-100%,not_reached
4,JG04,1.83,PG03,1.83,2.08,RM_type_A,Permanent,Male,1984,Y,...,11,2,1,1,95.17,100.72,2.0,achiev_< 50%,achiev_> 1.5,not_reached


## Data

Pada dataset ini saya menentukan menggunakan:   
* Fitur = Avg_achievement_%,	Last_achievement_%,	Achievement_above_100%_during3quartal,	achievement_target_1,	achievement_target_2,	achievement_target_3   
* Target = Best Performance  


In [3]:
dfx=df[['Best Performance',
       'Avg_achievement_%',
       'Last_achievement_%',
       'Achievement_above_100%_during3quartal',
       'achievement_target_1',
       'achievement_target_2',
       'achievement_target_3']]
dfx.head()

,Best Performance,Avg_achievement_%,Last_achievement_%,Achievement_above_100%_during3quartal,achievement_target_1,achievement_target_2,achievement_target_3
0,0,68.85,72.53,0.0,achiev_50%-100%,achiev_< 50%,reached
1,0,94.81,101.13,3.0,achiev_50%-100%,achiev_50%-100%,reached
2,0,72.82,71.30,0.0,achiev_100%-150%,achiev_< 50%,not_reached
3,0,86.89,90.09,2.0,achiev_50%-100%,achiev_50%-100%,not_reached
4,1,95.17,100.72,2.0,achiev_< 50%,achiev_> 1.5,not_reached


Mengubah penamaan kategori dalam fitur achievement_target_1, achievement_target_2, achievement_target_3

In [4]:
dfx['achievement_target_1']
for r in (('Pencapaian 50%-100%','achiev_50%-100%'),
          ('Pencapaian 100%-150%','achiev_100%-150%'),
          ('Pencapaian < 50%','achiev_< 50%'),
          ('Pencapaian > 1.5','achiev_> 1.5'),
          ('Tidak diberikan target','no_target')):
    df['achievement_target_1'] = df['achievement_target_1'].replace(*r)

In [5]:
dfx['achievement_target_2']
for r in (('Pencapaian 100%-150%','achiev_100%-150%'),
          ('Pencapaian 50%-100%','achiev_50%-100%'),
          ('Pencapaian < 50%','achiev_< 50%'),
          ('Pencapaian > 1.5','achiev_> 1.5'),
          ('Tidak diberikan target','no_target')):
    df['achievement_target_2'] = df['achievement_target_2'].replace(*r)

In [6]:
dfx['achievement_target_3'].replace('not reached','not_reached',inplace=True)

C:\Users\asus\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


> ## Missing Value

In [7]:
dfx.isna().sum()/df.shape[0]*100

Best Performance                          0.000000
Avg_achievement_%                        28.579868
Last_achievement_%                       28.638946
Achievement_above_100%_during3quartal    28.638946
achievement_target_1                     30.570325
achievement_target_2                     30.570325
achievement_target_3                     30.570325
dtype: float64

1. Missing value menggunakan ``iterative imputer`` untuk missing value data numerik pada fitur  Avg_achievement_%, Last_achievement_%, Achievement_above_100%_during3quartal

In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline

In [9]:
imp_int= IterativeImputer(max_iter=10, random_state=2020)
dfx[['Achievement_above_100%_during3quartal',
    'Last_achievement_%',
    'Avg_achievement_%',]]= imp_int.fit_transform(df[['Achievement_above_100%_during3quartal',
                                                      'Last_achievement_%',
                                                      'Avg_achievement_%',]])

<ipython-input-9-9b9a8158c409>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[['Achievement_above_100%_during3quartal',
C:\Users\asus\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [10]:
dfx.isna().sum()/df.shape[0]*100

Best Performance                          0.000000
Avg_achievement_%                         0.000000
Last_achievement_%                        0.000000
Achievement_above_100%_during3quartal     0.000000
achievement_target_1                     30.570325
achievement_target_2                     30.570325
achievement_target_3                     30.570325
dtype: float64

2. Missing value menggunakan ``simple imputer`` pada data kategorial pada fitur achievement_target_1, achievement_target_2, achievement_target_3  
* modus = achievement_target_1, achievement_target_2, achievement_target_3  
* one hot = achievement_target_3  
* ordinal = achievement_target_1, achievement_target_2

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import category_encoders as ce

In [12]:
#pipeline
#kolom achievement target 3
imputer_a=SimpleImputer(strategy='most_frequent')
pipe_scale_a=Pipeline([
    ('impute a', imputer_a),
    ('one hot', OneHotEncoder())
])

#kolom achievement target 1 dan 2
imputer_b=SimpleImputer(strategy='most_frequent')
mapping=[{'col':'achievement_target_2', 'mapping':{None: 0,
                                                    'achiev_> 1.5':1,
                                                    'achiev_< 50%':2,
                                                    'achiev_50%-100%':3,
                                                    'achiev_100%-150%':4,
                                                    'no_target':5}},
         {'col':'achievement_target_1', 'mapping':{None: 0,
                                                    'achiev_> 1.5':1,
                                                    'achiev_< 50%':2,
                                                    'achiev_50%-100%':3,
                                                    'achiev_100%-150%':4,
                                                    'no_target':5}}
        ]
ordinal=ce.OrdinalEncoder(mapping=mapping)
pipe_scale_b=Pipeline([
    ('impute b',imputer_b),
    ('ordinal', ordinal)
])

In [13]:
#transform
transformer= ColumnTransformer([
    ('One Hot', pipe_scale_a, ['achievement_target_3']),
    ('Ordinal', pipe_scale_b,['achievement_target_2','achievement_target_1',])
], remainder='passthrough')

In [14]:
dfx.isna().sum()/df.shape[0]*100

Best Performance                          0.000000
Avg_achievement_%                         0.000000
Last_achievement_%                        0.000000
Achievement_above_100%_during3quartal     0.000000
achievement_target_1                     30.570325
achievement_target_2                     30.570325
achievement_target_3                     30.570325
dtype: float64